In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sec_edgar.data.preprocessing.preprocess_4form_files import pre_process_4form_archive_files
from sec_edgar.data.preprocessing.preprocess_master_idx import pre_process_master_idx_content
from sec_edgar.data.processing.process_4form_files import Process4FormFiles
from sec_edgar.data.processing.process_append_historical_prices import ProcessAppendHistoricalPrices
from sec_edgar.data.utils.cik_mapping_util import CikMappingUtil
from sec_edgar.stock_historical_data.av_historical_data_request import AVHistoricalDataRequest
import pandas as pd

from sec_edgar.utils.evaluation_metrics import PerformanceMetric01, PerformanceMetric02, PerformanceMetric03
from sec_edgar.utils.performance_evaluations import PerformanceEvaluations4Form, PerformanceEvaluations4FormYear

In [2]:
pd.set_option('display.width', 2500)      # max table width to display

In [3]:
## Loading of data set

In [4]:
data_set_name = "us_market_allderv_AandB_1019.py"
base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

path_processed_datasets = base_path + '/Data/processed_datasets/'
file_name = path_processed_datasets + data_set_name + ".csv"

processed_4form_df = pd.read_csv(file_name, index_col=0)

In [5]:
print(file_name)
display(processed_4form_df)
display(processed_4form_df.columns.to_list())

C:\Users\andre\OneDrive\Documents\MathModels\sec_edgar/Data/processed_datasets/us_market_allderv_AandB_1019.py.csv


,CIK,Company_Names,Date_Filed,index_id,index_id_min,n_transactions,shares_percent_changes_mean,transactionSharesAdjust,date_ft_delta_mean,Filename,...,Shifted Price pct_change (10)(10 days 00:00:00),Shifted Price pct_change (10)(-10 days +00:00:00),Shifted Price pct_change (15)(15 days 00:00:00),Shifted Price pct_change (15)(-15 days +00:00:00),Shifted Price pct_change (21)(21 days 00:00:00),Shifted Price pct_change (21)(-21 days +00:00:00),Shifted Price pct_change (31)(31 days 00:00:00),Shifted Price pct_change (31)(-31 days +00:00:00),Shifted Price pct_change (42)(42 days 00:00:00),Shifted Price pct_change (42)(-42 days +00:00:00)
index,,,,,,,,,,,,,,,,,,,,,
48441,27904,DELTA AIR LINES INC /DE/,2010-01-04,[140546],140546,1,NaN,0.00,0 days 00:00:00,['edgar/data/27904/0001019687-10-000008.txt'],...,-0.071374,0.086661,-0.049117,0.075472,-0.043511,0.187556,0.140489,0.352113,0.174694,0.701164
107523,935703,DOLLAR TREE INC,2010-01-04,[578414 578415 578416 578417 578418],578414,5,0.250942,3544.61,3 days 00:00:00,['edgar/data/935703/0001127602-10-000234.txt'\...,...,0.005482,0.000612,-0.034031,0.014925,0.143033,0.017983,0.211652,0.053858,0.232375,-0.000204
48728,28412,COMERICA INC /NEW/,2010-01-04,[79905 79906 79907 79908 79909 79910 79911 799...,79905,13,0.018748,1152.00,5 days 00:00:00,['edgar/data/28412/0000028412-10-000001.txt'\n...,...,0.011134,0.068302,0.030569,0.152212,0.014648,0.167237,0.143071,0.266620,0.203143,0.213538
75418,72741,NORTHEAST UTILITIES,2010-01-04,[378340 378341 378342 378343],378340,4,0.146730,9435.00,0 days 00:00:00,['edgar/data/72741/0000072741-10-000004.txt'\n...,...,-0.040546,-0.000762,-0.051874,0.003069,0.028191,0.022868,0.074148,0.055285,0.109829,0.049897
97444,882835,ROPER INDUSTRIES INC,2010-01-04,[229120 229121 229122 229123],229120,4,-0.027286,0.00,5 days 00:00:00,['edgar/data/882835/0001209191-10-000310.txt'],...,-0.046095,-0.010336,-0.029718,-0.028972,0.074132,-0.028318,0.083082,-0.027359,0.165974,-0.048330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61641,49071,HUMANA INC,2019-12-31,[795842 795844 795845 795846 795847 795848 795...,795842,14,NaN,0.00,0 days 00:00:00,['edgar/data/49071/0000049071-19-000146.txt'\n...,...,-0.047265,-0.005206,-0.013514,0.024926,0.016773,0.056768,-0.189329,0.066934,-0.024946,0.088031
3960,1018724,AMAZON COM INC,2019-12-31,[334103],334103,1,NaN,0.00,4 days 00:00:00,['edgar/data/1018724/0000899243-19-030277.txt'],...,-0.029111,0.067372,0.101078,0.069387,0.138002,0.082767,-0.100004,0.033996,0.008402,0.076832
65091,54480,KANSAS CITY SOUTHERN,2019-12-31,[827929 827930 827932 827933 827934 827935 827...,827929,10,-0.001570,-285.00,11 days 00:00:00,['edgar/data/54480/0000054480-19-000247.txt'],...,0.036840,0.001612,0.076889,0.037715,0.063520,0.046904,-0.281522,0.047828,-0.197721,0.069084


['CIK',
 'Company_Names',
 'Date_Filed',
 'index_id',
 'index_id_min',
 'n_transactions',
 'shares_percent_changes_mean',
 'transactionSharesAdjust',
 'date_ft_delta_mean',
 'Filename',
 'Form_Type',
 'conversionOrExercisePrice',
 'date_ft_delta',
 'deemedExecutionDate',
 'directOrIndirectOwnership',
 'doc_name',
 'exerciseDate',
 'expirationDate',
 'file_url',
 'isDirector',
 'isOfficer',
 'isOther',
 'isTenPercentOwner',
 'issuerName',
 'issuerTradingSymbol',
 'officerTitle',
 'otherText',
 'periodOfReport',
 'quarter',
 'rptOwnerName',
 'securityTitle',
 'sharesOwnedFollowingTransaction',
 'shares_percent_changes',
 'signatureDate',
 'transactionAcquiredDisposedCode',
 'transactionCode',
 'transactionDate',
 'transactionFormType',
 'transactionPricePerShare',
 'transactionShares',
 'transactionTotalValue',
 'transaction_table',
 'transaction_type',
 'underlyingSecurityShares',
 'underlyingSecurityTitle',
 'underlyingSecurityValue',
 'valueOwnedFollowingTransaction',
 'year',
 'my_de

## Performance Metrics Evaluations

In [6]:
pe_4form = PerformanceEvaluations4Form(processed_4form_df, n_sub_set=5)

In [7]:
days = 15
column_label_eval = 'Price pct_change ({days})'.format(days=days)
shifted_columns_label_eval = 'Shifted Price pct_change ({days})(-{days} days +00:00:00)'.format(days=days)

In [8]:

# PerformanceMetric01 evaluation:
metrics_df_01, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric01,
                                        pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation: ")
display(metrics_df_01)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_02b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation: ")
display(metrics_df_02)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_03b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation: ")
display(metrics_df_03)
print("-" * 75)


PerformanceMetric01 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.009005,0.012037,0.006687,0.008284,0.008396,0.009623
benchmark,0.007062,0.008132,0.006704,0.006921,0.006104,0.007448
is_greater_prob,0.621176,0.752941,0.517647,0.611765,0.6,0.623529
is_pos_tse_ok,True,True,False,True,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,False,True,True,True
Positive_tsa_std,0.012952,0.009925,0.01142,0.010195,0.012654,0.018376
benchmark_std,0.007632,0.006062,0.007035,0.006504,0.005863,0.011345
statistic,2.665329,3.095635,-0.011727,1.038851,1.515472,0.928523
pvalue,0.003936,0.001188,0.50467,0.150316,0.066158,0.177368


---------------------------------------------------------------------------

PerformanceMetric02 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.009005,0.012037,0.006687,0.008284,0.008396,0.009623
benchmark,0.00651,0.008134,0.003817,0.009249,0.006414,0.004937
is_greater_prob,0.6,0.670588,0.623529,0.529412,0.588235,0.588235
is_pos_tse_ok,True,True,True,False,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,True,False,True,True
Positive_tsa_std,0.012952,0.009925,0.01142,0.010195,0.012654,0.018376
benchmark_std,0.016494,0.019383,0.011642,0.013503,0.013593,0.021791
statistic,2.45298,1.652825,1.62232,-0.525776,0.984143,1.51551
pvalue,0.00719,0.050436,0.053305,0.700105,0.163234,0.065788


---------------------------------------------------------------------------

PerformanceMetric03 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.008913,0.011323,0.007491,0.008029,0.007873,0.009495
benchmark,0.006601,0.007873,0.004416,0.008415,0.006328,0.005527
is_greater_prob,0.502028,0.511691,0.502284,0.488962,0.494641,0.512662
is_pos_tse_ok,True,True,True,False,True,True
is_greater_prob_ok,True,True,True,False,False,True
is_performance_ok,True,True,True,False,False,True
Positive_tsa_std,0.06926,0.077357,0.065842,0.068576,0.064044,0.066312
benchmark_std,0.093682,0.139057,0.067918,0.074135,0.070405,0.078488
statistic,2.940276,1.61029,2.306882,-0.258694,0.991968,2.143189
pvalue,0.001641,0.053686,0.010541,0.602061,0.160623,0.016069


---------------------------------------------------------------------------


In [9]:
print("=" * 90)
pe_4form_year = PerformanceEvaluations4FormYear(processed_4form_df)

# PerformanceMetric01 evaluation:
metrics_df_01_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric01,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation (Year): ")
display(metrics_df_01_year)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_02b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation (Year): ")
display(metrics_df_02_year)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_03b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation (Year): ")
display(metrics_df_03_year)
print("-" * 75)

print("=" * 90)


PerformanceMetric01 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.009005,0.0147,0.002978,0.007931,0.018349,0.007602,-0.002472,0.01961,0.010213,0.00326,0.01243
benchmark,0.007062,0.011146,-0.002583,0.009379,0.011831,0.007592,-0.006478,0.015649,0.009657,-0.003572,0.009158
is_greater_prob,0.621176,0.56599,0.581454,0.463476,0.608479,0.53616,0.611247,0.558252,0.5,0.577938,0.553699
is_pos_tse_ok,True,True,True,False,True,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,False,True,True,True,True,False,True,True
is_performance_ok,True,True,True,False,True,True,True,True,False,True,True
Positive_tsa_std,0.012952,0.042192,0.044606,0.03227,0.035175,0.053491,0.037783,0.041186,0.030597,0.034336,0.038866
benchmark_std,0.007632,0.02346,0.024472,0.018183,0.018247,0.017165,0.018818,0.028207,0.020761,0.020769,0.022229
statistic,2.665329,1.461188,2.183238,-0.778692,3.293837,0.003442,1.919494,1.610718,0.306509,3.476747,1.496079
pvalue,0.003936,0.072237,0.014697,0.781772,0.000523,0.498627,0.027698,0.053838,0.379652,0.00027,0.067554


---------------------------------------------------------------------------

PerformanceMetric02 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.009005,0.0147,0.002978,0.007931,0.018349,0.007602,-0.002472,0.01961,0.010213,0.00326,0.01243
benchmark,0.00651,0.014143,0.002462,0.010727,0.016566,0.014747,-0.002613,0.01291,0.010827,-0.008649,0.011699
is_greater_prob,0.6,0.517766,0.54386,0.455919,0.533666,0.413965,0.530562,0.541262,0.471154,0.585132,0.534606
is_pos_tse_ok,True,True,True,False,True,False,True,True,False,True,True
is_greater_prob_ok,True,True,True,False,True,False,True,True,False,True,True
is_performance_ok,True,True,True,False,True,False,True,True,False,True,True
Positive_tsa_std,0.012952,0.042192,0.044606,0.03227,0.035175,0.053491,0.037783,0.041186,0.030597,0.034336,0.038866
benchmark_std,0.016494,0.043215,0.08629,0.03756,0.035751,0.035801,0.037123,0.039077,0.036623,0.044472,0.04025
statistic,2.45298,0.183039,0.106195,-1.125187,0.711958,-2.222795,0.053755,2.395609,-0.262305,4.328461,0.267318
pvalue,0.00719,0.427407,0.457732,0.869571,0.238349,0.986726,0.478572,0.008408,0.603423,0.000008,0.394645


---------------------------------------------------------------------------

PerformanceMetric03 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.008913,0.015291,0.003133,0.005907,0.017078,0.009129,-0.002725,0.018631,0.008924,0.001919,0.012129
benchmark,0.006601,0.014891,0.000894,0.008744,0.014136,0.013741,-0.005831,0.010334,0.008196,-0.008003,0.009837
is_greater_prob,0.502028,0.495976,0.506932,0.487758,0.499516,0.449497,0.525129,0.51008,0.503484,0.538728,0.499133
is_pos_tse_ok,True,True,True,False,True,False,True,True,True,True,True
is_greater_prob_ok,True,False,True,False,False,False,True,True,True,True,False
is_performance_ok,True,False,True,False,False,False,True,True,True,True,False
Positive_tsa_std,0.06926,0.07586,0.103476,0.061741,0.054953,0.056605,0.063105,0.07335,0.050789,0.064316,0.066134
benchmark_std,0.093682,0.077074,0.207701,0.066124,0.05673,0.058231,0.070606,0.071552,0.064838,0.07306,0.071882
statistic,2.940276,0.179882,0.463604,-1.43144,1.692607,-2.595778,1.513733,3.73956,0.410243,4.900394,1.126603
pvalue,0.001641,0.428626,0.321481,0.92381,0.045303,0.995265,0.065084,0.000093,0.340824,0.0,0.129985


---------------------------------------------------------------------------


## OutLayers Rem Evaluation test

In [10]:
post_processed_4form_df = processed_4form_df.copy()
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df.my_derivative_types.isin(['AB', 'B'])]
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df['transactionSharesAdjust'] > 0]

In [11]:
def _ecdf(sample):
    sample = np.atleast_1d(sample)
    quantiles, counts = np.unique(sample, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob


def _get_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[np.logical_and(cumprob > 0.5 * ci, cumprob < 1. - 0.5 * ci)]

def _get_left_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[cumprob < 1. - ci]

quantiles, cumprob = _ecdf(post_processed_4form_df[column_label_eval].values)
results_dict = dict()
results_dict["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

quantiles, cumprob = _ecdf(post_processed_4form_df[shifted_columns_label_eval].values)
results_dict_bench = dict()
results_dict_bench["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict_bench["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict_bench["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict_bench["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

In [12]:
print("\n MostProb Scenarios results: \n")
display(pd.Series(results_dict))
print("\n MostProb Scenarios results Bench: \n")
display(pd.Series(results_dict_bench))


 MostProb Scenarios results: 



E[R|0.05<P(R)<0.95]    0.009261
E[R|0.10<P(R)<0.90]    0.009377
E[R|0.15<P(R)<0.85]    0.009453
E[R|0.20<P(R)<0.80]    0.009560
dtype: float64


 MostProb Scenarios results Bench: 



E[R|0.05<P(R)<0.95]    0.006566
E[R|0.10<P(R)<0.90]    0.006913
E[R|0.15<P(R)<0.85]    0.007232
E[R|0.20<P(R)<0.80]    0.007518
dtype: float64